## Exploratory Data Analysis - Locales en Venta en CABA

#### Importamos librerías relevantes
Pandas para manejo de data, numpy para operaciones matemáticas, matplolib y seaborn para graficar.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#### Locales en venta en CABA - 2020
##### Pre-procesamiento
El objetivo de este trabajo es predecir el precio de un local en la Ciudad de Buenos Aires a partir de la data disponible en el portal de información del Gobierno de la Ciudad para el 2020: https://data.buenosaires.gob.ar/dataset/locales-en-venta
Para lograr esto, en primera instancia se analizará este dataset y se lo limpiará en caso de que sea necesario.

Leemos el dataset y lo almacenamos en un dataframe:

In [ ]:
df = pd.read_csv('https://cdn.buenosaires.gob.ar/datosabiertos/datasets/secretaria-de-desarrollo-urbano/locales-en-venta/locales-en-venta-2020.csv')
print('Hay un total de {} filas en el base de datos.'.format(len(df.index)))
df.head()

Pasamos las columnas a minúscula y renombramos algunos nombres.

In [ ]:
df.columns = map(str.lower, df.columns)
df.rename(columns={"propiedads": "m2",
                   "en_galeria": "galeria",
                   "cotiz_": "dolar",
                   "barrios_": "barrios",
                   "comuna_": "comuna",
                  },
          inplace=True)
df.head()

In [ ]:
pd.set_option('display.float_format', lambda x: '%.0f' % x)
df[['m2', 'preciousd', 'usdm2', 'antig']].describe().loc[['mean', 'std', 'min', 'max']]
# Por si necesitamos resetear el formato
# pd.reset_option('display.float_format')

Se pueden ver las siguientes columnas:

In [ ]:
print(list(df))

Se descartan las columnas irrelevantes:
- direccion
- preciopeso
- pesosm2
- dolar
- trimestre_

In [ ]:
df = df.drop(axis=1, columns=['direccion', 'preciopeso', 'pesosm2', 'dolar', 'trimestre_'])
df.head()

Detectamos si los barrios y la variable galeria tienen datos consistentes y buscamos posibles errores. También buscamos valores 

In [ ]:
print('Barrios ordenados: ' + str(sorted(df.barrios.unique())))
print('Valores de galeria: ' + str(df.galeria.unique()))
df.isna().sum()

Los barrios no presentan inconvenientes, pero hay una cantidad importante de valores vacíos en galería, y además hay valores en galería iguales a 'S1' en lugar de 'SI'. Asumiremos que los valores vacíos representan locales que no se encuentran en galerías.

In [ ]:
df.galeria.fillna('NO', inplace=True)
df.galeria.replace(to_replace='S1', value='SI', inplace=True)
print('Valores de galeria: ' + str(df.galeria.unique()))
show=pd.DataFrame({
    'variable':df.isnull().sum().index,
    'na':df.isna().sum().values,
    'null':df.isnull().sum().values
})
show

Chequeamos que los valores sean del tipo correcto.

In [ ]:
df.dtypes

#### Análisis exploratorio
Graficamos precio promedio por barrio

In [ ]:
sns.set_context(context="paper", )
sns.set_theme(style="darkgrid")
plt.figure(figsize=(15,6))
sns.countplot(x=df.barrios.sort_values(ascending = True))
plt.title("Locales por barrio")
plt.ylabel("Cantidad de locales")
plt.xlabel("")
plt.xticks(rotation=90)
plt.show()

, por comuna, ver si aumenta el precio por estar en galeria o no.